# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import sys
sys.path.append('../../../')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,f64,str,str,str,str,bool
"""../../../../da…",1.2927e19,"""15130666264550…","""62024094566666…","""49372458409459…","""14011775049326…",false
"""../../../../da…",1.4856e19,"""17672222777748…","""69919860245105…","""49372458409459…","""11273311348027…",true
"""../../../../da…",5.6458e17,"""17733443242346…","""12718217377296…","""92464972637050…","""14011775049326…",false
"""../../../../da…",4.0723e18,"""10119956405041…","""69919860245105…","""21402025580214…","""14011775049326…",false
"""../../../../da…",1.4677e19,"""42453109902186…","""62024094566666…","""21402025580214…","""48377138514622…",true
"""../../../../da…",1.6330e19,"""18545686467940…","""12718217377296…","""42749569485692…",null,false
"""../../../../da…",8.7675e18,"""85918468513500…","""12718217377296…","""21402025580214…","""48377138514622…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_CLV': 'PREDICTOR_1',
 'Customer_City': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder='../../../../data/',
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",1.2927e19,"""26966398562794…","""58205489615742…","""FirstMortgage3…","""11001468374950…","""Rejected"""
"""../../../../da…",1.4856e19,"""10888342417233…","""17313535397387…","""FirstMortgage3…","""26645775286148…","""Accepted"""
"""../../../../da…",5.6458e17,"""70600923917482…","""17320622074982…","""MoneyMarketSav…","""11001468374950…","""Rejected"""
"""../../../../da…",4.0723e18,"""60228505380642…","""17313535397387…","""BasicChecking""","""11001468374950…","""Rejected"""
"""../../../../da…",1.4677e19,"""71350791186337…","""58205489615742…","""BasicChecking""","""14996720860198…","""Accepted"""
"""../../../../da…",1.6330e19,"""80164227603752…","""17320622074982…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",8.7675e18,"""64889919967575…","""17320622074982…","""BasicChecking""","""14996720860198…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str,str
"""../../../../da…",1.2927e19,"""53143111629009…","""46700455255518…","""FirstMortgage3…","""17018233096625…","""Rejected"""
"""../../../../da…",1.4856e19,"""25091924879780…","""70214574306593…","""FirstMortgage3…","""79064648968386…","""Accepted"""
"""../../../../da…",5.6458e17,"""88658368720705…","""80044616806818…","""MoneyMarketSav…","""17018233096625…","""Rejected"""
"""../../../../da…",4.0723e18,"""77105391344753…","""70214574306593…","""BasicChecking""","""17018233096625…","""Rejected"""
"""../../../../da…",1.4677e19,"""10172997720118…","""46700455255518…","""BasicChecking""","""63524210260497…","""Accepted"""
"""../../../../da…",1.6330e19,"""91880426861611…","""80044616806818…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…",8.7675e18,"""14720279436419…","""80044616806818…","""BasicChecking""","""63524210260497…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_CLV=PREDICTOR_1
Customer_City=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""66152841175816…","""16762902773178…","""FirstMortgage3…","""14961179910371…","""Rejected"""
1.4856e19,"""13507418157907…","""16265842560315…","""FirstMortgage3…","""17799528202669…","""Accepted"""
5.6458e17,"""16313753158498…","""15732322579337…","""MoneyMarketSav…","""14961179910371…","""Rejected"""
4.0723e18,"""14262518519424…","""16265842560315…","""BasicChecking""","""14961179910371…","""Rejected"""
1.4677e19,"""83187088782301…","""16762902773178…","""BasicChecking""","""15691861758992…","""Accepted"""
1.6330e19,"""93098615113191…","""15732322579337…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""65704538171004…","""15732322579337…","""BasicChecking""","""15691861758992…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'